<a href="https://colab.research.google.com/github/sahniaditya007/Crypto-Bot/blob/harshit/p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install numpy pandas scikit-learn tensorflow python-binance


In [25]:
import numpy as np
import pandas as pd
import time
import os
from binance.client import Client
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Input, Bidirectional, Dropout, Attention
from tensorflow.keras.optimizers import Adam

In [14]:
# Import and Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define Model Save Path in Google Drive
MODEL_PATH = "/content/drive/MyDrive/TradingBot/lstm_trading_model.h5"
print(f"Model will be saved at: {MODEL_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model will be saved at: /content/drive/MyDrive/TradingBot/lstm_trading_model.h5


In [6]:
# Binance Testnet API Credentials (Replace with your own keys from Testnet)
from binance.client import Client

API_KEY = "SqQl1AFmjzYiIKvnt1QQoSb8MUn5ussbH1KgkRZiAuHbxCR0SUfeM4SkOHOywU9G"
API_SECRET = "kIEa5qbDAf1LVHO84orgNPR5ADqx6pJwcPNgFdlvxKIv9r2E7Ibpuhc1M1DTqlSn"
client = Client(API_KEY, API_SECRET, tld='com', testnet=True)

In [7]:
# Function to fetch historical OHLCV data
def get_historical_data(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_1MINUTE, lookback='365 days ago UTC'):
    try:
        klines = client.get_historical_klines(symbol, interval, lookback)
        df = pd.DataFrame(klines, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
                                           'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
                                           'Taker buy quote asset volume', 'Ignore'])
        df = df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
        return df
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return None

data = get_historical_data()
if data is None:
    raise RuntimeError("Failed to fetch historical data!")

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [8]:
# Function to create time-series sequences
def create_sequences(data, time_steps=60):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps, 3])  # Predicting 'Close' price
    return np.array(X), np.array(y)

TIME_STEPS = 60
X, y = create_sequences(data_scaled, TIME_STEPS)
X_train, X_test, y_train, y_test = X[:int(0.8*len(X))], X[int(0.8*len(X)):], y[:int(0.8*len(y))], y[int(0.8*len(y)):]


In [9]:
# Build Advanced LSTM Model
def build_model():
    inputs = Input(shape=(TIME_STEPS, 5))
    x = Bidirectional(LSTM(128, return_sequences=True))(inputs)
    x = Dropout(0.2)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    attention = Attention()([x, x])
    x = Bidirectional(LSTM(64, return_sequences=False))(attention)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='linear')(x)
    model = tf.keras.Model(inputs, output)
    model.compile(optimizer=Adam(learning_rate=0.001, decay=1e-6), loss='mse')
    return model

In [26]:
# Load or Train Model
if os.path.exists(MODEL_PATH):
    model = load_model(MODEL_PATH)
    print("Model loaded successfully!")
else:
    model = build_model()
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
    model.save(MODEL_PATH)
    print(f"Model trained and saved at: {MODEL_PATH}")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


791/791 ━━━━━━━━━━━━━━━━━━━━ 26s 24ms/step - loss: 0.0161 - val_loss: 3.6530e-04
Epoch 2/50
791/791 ━━━━━━━━━━━━━━━━━━━━ 38s 23ms/step - loss: 0.0017 - val_loss: 2.5771e-05
Epoch 3/50
791/791 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - loss: 0.0016 - val_loss: 8.8654e-05
Epoch 4/50
791/791 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - loss: 0.0012 - val_loss: 3.5215e-04
Epoch 5/50
791/791 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - loss: 0.0010 - val_loss: 8.8676e-06
Epoch 6/50
791/791 ━━━━━━━━━━━━━━━━━━━━ 22s 23ms/step - loss: 9.7751e-04 - val_loss: 1.9571e-05
Epoch 7/50
791/791 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - loss: 6.4425e-04 - val_loss: 1.4244e-04
Epoch 8/50
791/791 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - loss: 4.7515e-04 - val_loss: 2.1878e-05
Epoch 9/50
791/791 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - loss: 5.0352e-04 - val_loss: 4.6746e-05
Epoch 10/50
791/791 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - loss: 3.3414e-04 - val_loss: 9.5175e-06
Epoch 11/50
791/791 ━━━━━━━━━━━━━━━━━━━━ 21s 24ms/step - loss: 2.2663e

Model trained and saved at: /content/drive/MyDrive/TradingBot/lstm_trading_model.h5


In [31]:
def predict_next_1_minute(symbol='BTCUSDT'):
    try:
        latest_data = get_historical_data(symbol)  # Get last TIME_STEPS values
        latest_scaled = scaler.transform(latest_data)
        latest_scaled = np.expand_dims(latest_scaled, axis=0)

        prediction = model.predict(latest_scaled)  # Predict normalized close price
        predicted_prices = scaler.inverse_transform(
            np.hstack((latest_data[-1, :-1], prediction[0])).reshape(1, -1)
        )[:, -1]  # Extract predicted Close price

        return predicted_prices[0]
    except Exception as e:
        print(f"Error predicting next 1 minute: {e}")
        return None

In [42]:
def predict_next_1_minute(symbol='BTCUSDT'):
    try:
        latest_data = get_historical_data().values[-TIME_STEPS:]
        print(f"Latest Data Shape: {latest_data.shape}")  # Debugging

        if latest_data.shape[0] != TIME_STEPS:
            raise ValueError(f"Expected shape ({TIME_STEPS}, 5), but got {latest_data.shape}")

        latest_scaled = scaler.transform(latest_data)
        latest_scaled = np.expand_dims(latest_scaled, axis=0)
        print(f"Scaled Data Shape: {latest_scaled.shape}")  # Debugging

        prediction = model.predict(latest_scaled)
        print(f"Prediction Shape: {prediction.shape}")  # Debugging

        # Ensure the transformation aligns with training features
        predicted_prices = scaler.inverse_transform(
            np.hstack((np.zeros((1, latest_data.shape[1] - 1)), prediction.reshape(-1, 1)))
        )[:, -1]

        return predicted_prices
    except Exception as e:
        print(f"Error predicting next 1 minute: {e}")
        return None


In [43]:
# Running real-time prediction & trading loop (Simulated)
try:
    while True:
        execute_trade()
        time.sleep(10)  # Wait 1 minute before next execution
except KeyboardInterrupt:
    print("\nTrading bot stopped.")

Latest Data Shape: (60, 5)
Scaled Data Shape: (1, 60, 5)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Prediction Shape: (1, 1)
Predicted Price: [13.66528332], Current Price: 86377.71
Stop-Loss Price: 84650.15580000001, Take-Profit Price: 90696.59550000001

Trading bot stopped.
Enter 'BUY' or 'SELL' or 'SKIP': SKIP
